In [1]:
import rioxarray
import numpy as np

In [2]:
xds = rioxarray.open_rasterio("./dems/larimerAreaDEM.tif")

In [3]:
xds_wgs = xds.rio.reproject("wgs84")

In [4]:
xds_wgs.rio.bounds()

(-106.00055555555541, 39.99944444471632, -104.99944444471632, 41.0005555555554)

In [5]:
elevMap = xds_wgs.isel()[0].values

In [6]:
xVals = xds_wgs.x.isel().values
xStart = xVals[0]
xStep = (xVals[-1] - xVals[0]) / (len(xVals)-1)

yVals = xds_wgs.y.isel().values
yStart = yVals[0]
yStep = (yVals[-1] - yVals[0]) / (len(yVals)-1)

In [7]:
e = elevMap.astype('int16')

In [8]:
e

array([[2402, 2403, 2403, ..., 1965, 1966, 1966],
       [2401, 2402, 2403, ..., 1966, 1966, 1967],
       [2400, 2403, 2403, ..., 1967, 1967, 1968],
       ...,
       [2795, 2796, 2798, ..., 1588, 1588, 1588],
       [2796, 2798, 2799, ..., 1588, 1588, 1588],
       [2798, 2798, 2799, ..., 1588, 1588, 1588]], dtype=int16)

In [9]:
numSplits = 6

In [10]:
perMini = 10812 // numSplits
perMini

1802

In [11]:
def split(array, nrows, ncols):
    """Split a matrix into sub-matrices."""
    r, h = array.shape
    return (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))

In [12]:
m = split(e,perMini,perMini)
m

array([[[2402, 2403, 2403, ..., 2328, 2327, 2327],
        [2401, 2402, 2403, ..., 2328, 2328, 2328],
        [2400, 2403, 2403, ..., 2328, 2329, 2329],
        ...,
        [2936, 2935, 2935, ..., 3125, 3125, 3124],
        [2935, 2935, 2934, ..., 3125, 3125, 3124],
        [2934, 2934, 2934, ..., 3124, 3124, 3123]],

       [[2327, 2327, 2327, ..., 2598, 2599, 2600],
        [2328, 2327, 2327, ..., 2598, 2599, 2600],
        [2328, 2328, 2328, ..., 2598, 2599, 2600],
        ...,
        [3123, 3123, 3123, ..., 2743, 2745, 2747],
        [3123, 3123, 3122, ..., 2748, 2750, 2752],
        [3123, 3122, 3122, ..., 2753, 2755, 2758]],

       [[2601, 2601, 2602, ..., 2405, 2405, 2405],
        [2601, 2602, 2603, ..., 2406, 2406, 2406],
        [2601, 2603, 2604, ..., 2407, 2407, 2406],
        ...,
        [2750, 2752, 2754, ..., 2307, 2306, 2306],
        [2755, 2758, 2760, ..., 2306, 2306, 2306],
        [2761, 2765, 2766, ..., 2305, 2305, 2305]],

       ...,

       [[2790, 2789, 278

In [13]:
import json
for i,tile in enumerate(m):
    x = i%numSplits
    y = i//numSplits
    listOut = tile.tolist()
    d = {
        "x": x,
        "y": y,
        "stepX": xStep,
        "stepY": yStep,
        "startX": xStart + xStep * (perMini * x),
        "startY": yStart + yStep * (perMini * y),
        "vals": listOut
    }
    #jsonString = json.dumps(d)
    #jsonFile = open(f"jsonOut2/tile{x}X{y}.json", "w")
    #jsonFile.write(jsonString)
    #jsonFile.close()

In [1]:
import geohash

In [3]:
geohash.bbox('9xj6')

{'s': 39.7265625, 'w': -105.1171875, 'n': 39.90234375, 'e': -104.765625}